In [73]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime


# Create dependent variables


### 1. Create list of tickers

In [74]:
tickers = ["BTC-USD", "SPY", "GLD", "DX-Y.NYB"]
start = '2017-07-03'
end = '2022-09-25'

###  2. Create data frame to store weekly return for each ticker


[Retrieve financial data from yahoo finance](https://pypi.org/project/yfinance/)

[Deal with multi-level columns](https://stackoverflow.com/questions/63107594/how-to-deal-with-multi-level-column-names-downloaded-with-yfinance/63107801#63107801)

[Resample data using pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html)

[Function pct_change()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html)

In [76]:
# Retrieve daily data from yahoo finance from March 7, 2017 to September 25, 2022 (5 years)
# Resample into weekly data bases on average of that week
# Calculate percent change from last week and multiply by 100 to get (%) unit 
data = yf.download(tickers, 
                   start='2017-7-3', 
                   end='2022-9-25').resample('W').mean().pct_change(periods=1)*100 

# Take only the closing price
data = data['Close']   

#print(data)



[*********************100%***********************]  4 of 4 completed


## 3. Calculate rolling 12 weeks covariance between tickers

[Accessing the index in 'for' loops](https://stackoverflow.com/questions/522563/accessing-the-index-in-for-loops)

[Calculate rolling covariance: pandas.DataFrame.rolling](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html)

[Format string literal](https://stackoverflow.com/questions/57150426/what-is-printf)

[Selecting multiple columns in a Pandas dataframe](https://stackoverflow.com/questions/11285613/selecting-multiple-columns-in-a-pandas-dataframe/11287278#11287278)

In [78]:
for i, ticker1 in enumerate(tickers[:-1]):      # function enumerate() iterates through both index and value 
                                                # of the element in the list
    for ticker2 in tickers[i+1:]:       # Loop through the list starting from the next ticker to ticker 1
        data[f"{ticker1}-{ticker2}"] = data[ticker1].rolling(12).cov(data[ticker2])
 #name the column using formatted string literal 
                                       # Calculate rolling covariance between two tickers using 12-week window
                                       # Rolling function uses up to 11 weeks prior to calculate
        data = data.iloc[ : , :7]      # Update data frame to include only first 7 columns

#print(data)
        
        


# Create independent variable

2 steps:
- Step 1: Collect data from web or csv files
- Step 2: Combine data frames

[Import csv columns into data frame](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)

[Combine 2 data frame using concatnate](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)


### 1. Collect CBOE VIX data from yahoo finance

In [79]:
# Download data from yahoo finance
data_vix = yf.download('^VIX',
                     start='2017-7-3', 
                     end='2022-9-25').resample('W').mean()
data_vix = data_vix['Close']

#print(data4.head(15))

[*********************100%***********************]  1 of 1 completed


In [80]:
# Combine VIX into existing data frame
data3 = pd.concat([data, data_vix], axis=1)

data3.index = data3.index.date              # Extract the date part from datetime index
data3.index = pd.to_datetime(data3.index)   # Convert the date index back into datetime format
data3 = data3.groupby(data3.index).first().reset_index()  # Group by date and only take the first row
 
# print(data3)

### 2. Collect Bitcoin google trends and GARCH(1,1) volatility from csv files



In [81]:
# Import specific columns from csv file
data_additional = pd.read_csv('/Users/quanghuy/Documents/ECO 590 - R & Python/Nguyen_Huy_additionaldata.csv', 
                     delimiter = ';')

data_additional


gg_trend_wrld    btc_garch
0               .  53,62767006
1               .  73,76204382
2               .  128,6168412
3               .  97,60370461
4               .  82,27699884
..            ...          ...
268            17  56,24453807
269            18  45,31278637
270            17  55,57379792
271            17   73,3579912
272            18  52,57561772

[273 rows x 2 columns]

In [83]:
# Combine data frames
data4 = pd.concat([data3, data_additional], axis=1)

# Replace '.' value with none
data4 = data4.replace('.', None)    

# Fix wrong decimal seperator, from ',' to '.'. And convert the 'btc_garch' column to numeric value
data4['btc_garch'] = data4['btc_garch'].str.replace(',', '.').astype(float)

#  Convert the 'gg_trend_wrld' column to numeric value
data4['gg_trend_wrld'] = data4['gg_trend_wrld'].astype(float)

print(data4.dtypes)
# data4.head(15)

index               datetime64[ns]
BTC-USD                    float64
DX-Y.NYB                   float64
GLD                        float64
SPY                        float64
BTC-USD-SPY                float64
BTC-USD-GLD                float64
BTC-USD-DX-Y.NYB           float64
Close                      float64
gg_trend_wrld              float64
btc_garch                  float64
dtype: object


### 3. Create time dummy variable for covid period

In [85]:
data4['covid'] = 0

start_covid = '2020-01-01'
end_covid = '2020-08-31'

requirement = (data4['index'] >= start_covid) & (data4['index'] <= end_covid)
data4.loc[requirement,'covid'] = 1
#filtered_data = data4[data4['covid'] == 1]
#len(filtered_data)

# data4

# Export data frame into csv file

[Rename column names](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html#pandas.DataFrame.rename)

[Python to csv file](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)



In [98]:
# Rename columns
data4 = data4.rename(columns ={'index':'date','BTC-USD': 'r_btc(%)',
                           'DX-Y.NYB': 'r_usd(%)', 
                           'GLD': 'r_gld(%)',
                           'SPY': 'r_spy(%)',
                           'BTC-USD-SPY': 'cov_bspy',
                           'BTC-USD-GLD': 'cov_bgld',
                           'BTC-USD-DX-Y.NYB': 'cov_busd',
                           'Close': 'VIX'})


data4 = data4.dropna()
data4.describe()

r_btc(%)    r_usd(%)    r_gld(%)    r_spy(%)    cov_bspy    cov_bgld  \
count  255.000000  255.000000  255.000000  255.000000  255.000000  255.000000   
mean     0.826161    0.064820    0.110293    0.175030    6.399969    1.734035   
std      9.329991    0.697489    1.595377    2.162656   12.276907    6.006592   
min    -28.672252   -1.865016   -7.030331  -11.257790   -7.962287  -13.810481   
25%     -4.716526   -0.364113   -0.681328   -0.582352    0.122930   -1.213211   
50%      0.017075    0.070329    0.192673    0.536260    2.551028    1.120004   
75%      5.749000    0.475557    0.836090    1.407775    7.384092    3.669636   
max     40.335599    4.208329    7.448337    6.890572   57.548421   23.094400   

         cov_busd         VIX  gg_trend_wrld   btc_garch       covid  
count  255.000000  255.000000     255.000000  255.000000  255.000000  
mean    -0.924093   20.724039      21.043137   67.604661    0.137255  
std      2.003313    8.423487      15.373119   24.612861    0.344793  
min     -6.134117    9.340000       6.000000   36.744210    0.000000  
25%     -2.024301   14.928000      10.000000   51.652433    0.000000  
50%     -0.980785   19.166000      16.000000   60.068708    0.000000  
75%      0.073591   24.378750      27.000000   75.839375    0.000000  
max      6.377844   74.618001     100.000000  179.208595    1.000000

In [94]:
is_covid = data4[data4['covid'] == 1].iloc[:, 1:8].describe()
is_covid

r_btc(%)   r_usd(%)   r_gld(%)   r_spy(%)   cov_bspy   cov_bgld  \
count  35.000000  35.000000  35.000000  35.000000  35.000000  35.000000   
mean    1.633800  -0.130306   0.755234   0.280276  18.784489   8.353735   
std     7.977522   1.027842   2.457705   3.694023  23.531535   9.370050   
min   -27.193793  -1.865016  -7.030331 -11.257790  -1.263132  -3.028236   
25%    -1.622321  -0.772800  -0.299939  -0.251780   0.433278   0.552936   
50%     0.774315  -0.119679   0.565190   0.725986   3.388018   6.322294   
75%     4.773651   0.334083   1.839474   2.348023  48.349265  20.246555   
max    17.205678   4.208329   7.448337   6.890572  57.548421  23.094400   

        cov_busd  
count  35.000000  
mean   -1.399466  
std     2.213193  
min    -6.134117  
25%    -2.813532  
50%    -1.010290  
75%     0.592068  
max     2.188519

In [95]:
is_notcovid = data4[data4['covid'] == 0].iloc[:, 1:8].describe()
is_notcovid

r_btc(%)    r_usd(%)    r_gld(%)    r_spy(%)    cov_bspy    cov_bgld  \
count  220.000000  220.000000  220.000000  220.000000  220.000000  220.000000   
mean     0.697673    0.095862    0.007689    0.158287    4.429705    0.680901   
std      9.537269    0.627034    1.391828    1.817686    7.773733    4.482977   
min    -28.672252   -1.632574   -3.951900   -8.345362   -7.962287  -13.810481   
25%     -5.183101   -0.338010   -0.724019   -0.717362   -0.009101   -1.336174   
50%     -0.079203    0.077497    0.147928    0.471489    2.381051    0.957781   
75%      5.818910    0.528365    0.760986    1.321319    5.986790    3.192556   
max     40.335599    1.914949    3.805811    3.883240   33.850103   10.685418   

         cov_busd  
count  220.000000  
mean    -0.848465  
std      1.962737  
min     -6.080199  
25%     -1.922544  
50%     -0.962295  
75%      0.051628  
max      6.377844

In [97]:

# Export to csv file
data4.to_csv('Nguyen_Huy_Data.csv',index = False)
